In [1]:
import pandas as pd

data = "train.parquet"

df = pd.read_parquet(data)

In [2]:
freqs = df.freq.unique()

In [3]:
freq_x_horizon = {
'10T': 48,
'H': 48,
'5T': 48,
'W-SUN': 13,
'10S': 60,
'D': 14,
'W-FRI': 8,
'M': 12,
'Q-DEC': 8,
'15T': 48,
'W-THU': 8,
'A-DEC': 6,
'W-WED': 8,
'W-TUE': 8
}

In [4]:
df[df.freq == '5T']

,id,freq,start,dataset_id,target
0,e8378544f5a8f15eda80,5T,2015-01-01,6b9bb095c2,"[1.4271739, 1.5165688, 1.5904777, 1.5551335, 1..."
1,e8378544f5a5e0013237,5T,2015-01-01,6b9bb095c2,"[1.5603069, 1.4629052, 1.5205714, 1.5356064, 1..."
2,e8378544f57cbbc409ec,5T,2015-01-01,6b9bb095c2,"[1.5383402, 1.5424885, 1.4719441, 1.5120252, 1..."
3,e8378544f535f4a8d465,5T,2015-01-01,6b9bb095c2,"[1.4774168, 1.5940177, 1.5791488, 1.5298705, 1..."
4,e8378544f5a3c65c2974,5T,2015-01-01,6b9bb095c2,"[1.5834944, 1.6328278, 1.6062033, 1.6142886, 1..."
...,...,...,...,...,...
318,e8378544f5eda80a3d5b,5T,2015-01-01,6b9bb095c2,"[1.4170375, 1.5042562, 1.5225235, 1.5539571, 1..."
319,e8378544f5b53b3a3d6a,5T,2015-01-01,6b9bb095c2,"[1.3737901, 1.4500607, 1.4098532, 1.404394, 1...."
320,e8378544f5f340f1b1f6,5T,2015-01-01,6b9bb095c2,"[1.4774817, 1.5188916, 1.5200636, 1.5249575, 1..."
321,e8378544f5bcbe3365e6,5T,2015-01-01,6b9bb095c2,"[1.514647, 1.5088638, 1.5564455, 1.5605385, 1...."


In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from chronos import BaseChronosPipeline
from copy import deepcopy

TARGETS_COUNT = 500

pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-bolt-tiny",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

result = pd.DataFrame(columns=['id', 'forecast'])

for freq in freqs:
    df_view = deepcopy(df[df.freq == freq])
    
    print(f'freq = {freq}')
    
    prediction_length = freq_x_horizon[freq]
    
    for _, row in df_view.iterrows():
        
        target = row['target'].astype("float")

        context = torch.tensor(target[-TARGETS_COUNT:])
        
        id = row['id']

        forecast = pipeline.predict(context, prediction_length)
        
        result = pd.concat([result, pd.DataFrame({
            'id': [id],
            'forecast': [forecast.mean(dim=1)[0].tolist()]
        })], ignore_index=True)


result.to_csv("submission_4.csv", index=False)
        

freq = 5T
freq = D
freq = H
freq = 15T
freq = M
freq = W-FRI
freq = W-WED
freq = Q-DEC
freq = W-SUN
freq = A-DEC
freq = 10T
